## 第3章 加工原料文本

文本的最重要来源无疑是网络。探索现成的文本集合，如我们在前面章节中看到的语料 库，是很方便的。然而，在你心中可能有你自己的文本来源，需要学习如何访问它们。

本章的目的是要回答下列问题： 
1. 我们怎样才能编写程序访问本地和网络上的文件，从而获得无限的语言材料？ 
2. 我们如何把文档分割成单独的词和标点符号，这样我们就可以开始像前面章节中在文本语料上做的那样的分析？
3. 我们怎样编程程序产生格式化的输出，并把结果保存在一个文件中？ 

为了解决这些问题，我们将讲述NLP中的关键概念，包括**分词**和**词干提取**。在此过程中，你会巩固你的Python 知识并且了解关于字符串、文件和正则表达式知识。既然这些网络上的文本都是HTML格式的，我们也将看到如何去除HTML标记。

> 从本章开始往后我们的例子程序将假设你以下面的导入语句开始你的交互式会话或程序：`from __future__ import division`, `import nltk, re, pprint`

### 3.1 从网络和硬盘访问文本

#### 电子书

NLTK 语料库集合中有古腾堡项目的一小部分样例文本。然而，你可能对分析古腾堡项目的其它文本感兴趣。你可以在 `http://www.gutenberg.org/catalog/`上浏览 25,000本免费在线书籍的目录，获得`ASCII`码文本文件的`URL`。虽然90％的古腾堡项目的文本是英语的，它还包括超过50种语言的材料，包括加泰罗尼亚语、中文、荷兰语、芬兰语、法语、德语、意大利语、葡萄牙语和西班牙语（每种语言都有超过100个文本）。

编号2554的文本是《罪与罚》的英文翻译，我们可以如下方式访问它。

In [4]:
# from urllib import urlopen
from urllib.request import urlopen
# url = 'http://www.gutenberg.org/files/2554/2554.txt'
url = 'https://www.gutenberg.org/cache/epub/68664/pg68664.txt'
raw = urlopen(url).read()

In [5]:
type(raw)

bytes

**必须使用`decode('utf-8')`将`html`(作为字节对象获得)转换为字符串：**

In [6]:
raw = raw.decode('utf-8')

In [7]:
type(raw)

str

In [8]:
len(raw)

716082

In [9]:
raw[:75]

'\ufeffThe Project Gutenberg eBook of The heart of the railroad problem, by\r\nFran'

对于语言处理，我们要将字符串分解为词和标点符号，正如我们在第1章中所看到的。这一步被称为分词，它产生我们所熟悉的结构，一个词汇和标点符号的链表。

In [10]:
import nltk
# nltk.download() #设置host才能下载
# word_tokenize()需要下载punkt包
tokens = nltk.word_tokenize(raw)
type(tokens)

list

In [11]:
len(tokens)

135456

In [12]:
tokens[:10]

['\ufeffThe',
 'Project',
 'Gutenberg',
 'eBook',
 'of',
 'The',
 'heart',
 'of',
 'the',
 'railroad']

请注意，NLTK 需要分词，但所有前面的打开一个URL读入一个字符串的任务都没有分词。如果我们现在采取进一步的步骤从这个链表创建一个NLTK文本，我们可以进行我们在第1 章看到的所有的其他语言的处理，也包括常规的链表操作，例如切片：

In [13]:
text = nltk.Text(tokens)
type(text)

nltk.text.Text

In [14]:
text[1020:1060]

['METHODS',
 '159',
 'XXV',
 '.',
 'TERMINAL',
 'RAILROADS',
 '166',
 'XXVI',
 '.',
 'PRIVATE-CAR',
 'ABUSES',
 '174',
 'XXVII',
 '.',
 'THE',
 'LONG-HAUL',
 'ANOMALY',
 '208',
 'XXVIII',
 '.',
 'OTHER',
 'PLACE',
 'DISCRIMINATIONS',
 '216',
 'XXIX',
 '.',
 'NULLIFYING',
 'THE',
 'PROTECTIVE',
 'TARIFF',
 '221',
 'XXX',
 '.',
 'SUMMARY',
 'OF',
 'METHODS',
 'AND',
 'RESULTS',
 '228',
 'XXXI']

In [17]:
text.collocation_list()

['New York',
 'United States',
 'Sen. Com',
 'Interstate Commerce',
 'Supreme Court',
 'Project Gutenberg-tm',
 'per hundred',
 'Missouri River',
 'Kansas City',
 'San Francisco',
 'Colorado Fuel',
 'cents per',
 'Standard Oil',
 'St. Louis',
 'Circuit Court',
 'COMMISSIONER CLEMENTS',
 'Commerce Commission',
 'Project Gutenberg',
 'New Orleans',
 'Interstate Commission']

请注意，古腾堡项目以一个排列的形式出现。这是因为从古腾堡项目下载的每个文本都包含一个首部，里面有文本的名称、作者、扫描和校对文本的人的名字、许可证等信息。有 时这些信息出现在文件末尾页脚处。我们不能可靠地检测出文本内容的开始和结束，因此在从原始文本中挑出内容之前，我们需要手工检查文件以发现标记内容开始和结尾的独特的字符串。

In [18]:
raw.find("PART I")

691846

In [19]:
raw.rfind("End of Project Gutenberg's Crime")

-1

方法 `find()`和`rfind()`（反向的 `find`）帮助我们得到字符串切片需要用到的正确的索引值。

这是我们第一次接触到网络的实际内容：在网络上找到的文本可能含有不必要的内容， 并没有一个自动的方法来去除它。但只需要少量的额外工作，我们就可以提取出我们需要的 材料。

#### 处理的`HTML`

网络上的文本大部分是`HTML`文件的形式。你可以使用网络浏览器将网页作为文本保
存为本地文件，然后按照后面关于文件的小节描述的那样来访问它。不过，如果你要经常这样做，最简单的办法是直接让Python来做这份工作。第一步是像以前一样使用`urlopen`。 为了好玩，我们将挑选被称为“Blondes to die out in 200 years”的 BBC 新闻故事，一个都市传奇被BBC作为确立的科学事实流传下来。

In [24]:
from urllib.request import urlopen
# url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
url = "https://www.cnblogs.com/sonofelice/p/7890900.html"
html = urlopen(url).read()

In [25]:
html[:60]

b'<!DOCTYPE html>\n<html lang="zh-cn">\n<head>\n    <meta charset'

输入`print(html)` 可以看到HTML的全部内容，包括 `meta` 元标签、图像标签、`map`标签、`JavaScript`、表单和表格。 从`HTML`中提取文本是极其常见的任务，NLTK提供了一个辅助函数 `nltk.clean_html()`将 `HTML` 字符串作为参数，返回原始文本。然后我们可以对原始文本进行分词，获得我们熟悉的文本结构：

In [26]:
import nltk
# raw = nltk.clean_html(html)
# nltk已经不再支持clean_html()和clean_url()这两个函数，
# 需要得到相同功能的可以安装beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [28]:
text = soup.get_text()
print(type(text))
print(text)

<class 'str'>










python自然语言处理——学习笔记：Chapter3纠错 - SonoFelice - 博客园









        var currentBlogId = 224377;
        var currentBlogApp = 'sonofelice';
        var isLogined = false;
        var isBlogOwner = false;
        var skinName = 'coffee';
        var visitorUserId = '';
        var hasCustomScript = false;
        try {
            if (hasCustomScript && document.referrer && document.referrer.indexOf('baidu.com') >= 0) {
                Object.defineProperty(document, 'referrer', { value: '' });
                Object.defineProperty(Document.prototype, 'referrer', { get: function(){ return ''; } });
            }
        } catch(error) { }
        window.cb_enable_mathjax = false;
        window.mathEngine = 0;
        window.codeHighlightEngine = 1;
        window.enableCodeLineNumber = false;
        window.codeHighlightTheme = 'cnblogs';
        window.darkModeCodeHighlightTheme = 'vs2015';
        window.isDarkCodeHighlightTheme = false;
        window.isDarkMode

In [35]:
tokens = nltk.word_tokenize(text)
type(tokens)

TypeError: expected string or bytes-like object

In [30]:
print(tokens)

['python自然语言处理——学习笔记：Chapter3纠错', '-', 'SonoFelice', '-', '博客园', 'var', 'currentBlogId', '=', '224377', ';', 'var', 'currentBlogApp', '=', "'sonofelice", "'", ';', 'var', 'isLogined', '=', 'false', ';', 'var', 'isBlogOwner', '=', 'false', ';', 'var', 'skinName', '=', "'coffee", "'", ';', 'var', 'visitorUserId', '=', '``', ';', 'var', 'hasCustomScript', '=', 'false', ';', 'try', '{', 'if', '(', 'hasCustomScript', '&', '&', 'document.referrer', '&', '&', 'document.referrer.indexOf', '(', "'baidu.com", "'", ')', '>', '=', '0', ')', '{', 'Object.defineProperty', '(', 'document', ',', "'referrer", "'", ',', '{', 'value', ':', '``', '}', ')', ';', 'Object.defineProperty', '(', 'Document.prototype', ',', "'referrer", "'", ',', '{', 'get', ':', 'function', '(', ')', '{', 'return', '``', ';', '}', '}', ')', ';', '}', '}', 'catch', '(', 'error', ')', '{', '}', 'window.cb_enable_mathjax', '=', 'false', ';', 'window.mathEngine', '=', '0', ';', 'window.codeHighlightEngine', '=', '1', ';', 'window.e

其中仍然含有不需要的内容，包括网站导航及有关报道等。通过一些尝试和出错你可以找到内容索引的开始和结尾，并选择你感兴趣的标识符，按照前面讲的那样初始化一个文本。

In [36]:
tokens = tokens[36:100]
print(tokens)

['SonoFelice——棣琦', 'Never', 'give', 'up', '!', '博客园', '首页', '新随笔', '联系', '管理', '订阅', 'loadBlogStats', '(', ')', ';', 'python自然语言处理——学习笔记：Chapter3纠错', '2017-12-06更新：很多代码执行结果与书中不一致，是因为python的版本不一致。如果发现有问题，可以参考英文版：', 'p.p1', '{', 'margin', ':', '0', ';', 'font', ':', '12px', 'Arial', ';']


In [40]:
text = nltk.Text(tokens)
print(text)

<Text: SonoFelice——棣琦 Never give up ! 博客园 首页 新随笔...>


In [41]:
text.concordance('give')

Displaying 1 of 1 matches:
SonoFelice——棣琦 Never give up ! 博客园 首页 新随笔 联系 管理 订阅 loadBlogSta


更多更复杂的有关处理`HTML`的内容，可以使用 `http://www.crummy.com/software/BeautifulSoup/`上的`Beautiful Soup` 软件包。


#### 处理搜索引擎的结果

网络可以被看作未经标注的巨大的语料库。网络搜索引擎提供了一个有效的手段，搜索大量文本作为有关的语言学的例子。搜索引擎的主要优势是规模：因为你正在寻找这样庞大 的一个文件集，会更容易找到你感兴趣语言模式。而且，你可以使用非常具体的模式，仅仅在较小的范围匹配一两个例子，但在网络上可能匹配成千上万的例子。网络搜索引擎的第二个优势是非常容易使用。因此，它是一个非常方便的工具，可以快速检查一个理论是否合理。 请看表 3-1 的一个例子。
![list3-1](./imgs/list3-1.jpg)

不幸的是，搜索引擎有一些显著的缺点。首先，允许的搜索方式的范围受到严格限制。
不同于本地驱动器中的语料库，你可以编写程序来搜索任意复杂的模式，搜索引擎一般只允许你搜索单个词或词串，有时也允许使用通配符。其次，搜索引擎给出的结果不一致，并且在不同的时间或在不同的地理区域会给出非常不同的结果。当内容在多个站点重复时，搜索结果会增加。最后，搜索引擎返回的结果中的标记可能会不可预料的改变，基于模式的方法定位特定的内容将无法使用（通过使用搜索引擎APIs可以改善这个问题）


#### 处理RSS订阅

博客圈是文本的重要来源，无论是正式的还是非正式的。在一个叫做`Universal Feed Parser` 的第三方Python库（可从 `http://feedparser.org/`免费下载）的帮助下，我们可以访问一个博客的内容，如下所示：

In [9]:
import feedparser
# from urllib.request import ProxyHandler, build_opener
# proxy = ProxyHandler({"http":"proxy.example.com:8080"})
llog = feedparser.parse('http://languagelog.ldc.upenn.edu/nll/?feed=atom', handlers=[proxy])
# d = feedparser.parse("http://feedparser.org/docs/examples/atom10.xml")

In [7]:
llog.keys()

dict_keys(['bozo', 'entries', 'feed', 'headers', 'bozo_exception'])

In [8]:
llog

{'bozo': True,
 'entries': [],
 'feed': {},
 'headers': {},
 'bozo_exception': urllib.error.URLError(socket.gaierror(11001,
                                       'getaddrinfo failed'))}

In [47]:
len(llog.entries)

0

#### 读取本地文件

为了读取本地文件，我们需要使用Python 内置的`open()`函数，然后是 `read()`方法。 假设你有一个文件`document.txt`，你可以像这样加载它的内容：
```python
f = open('document.txt')
raw = f.read()
```

In [1]:
import os
os.listdir('.')

['ch1-语言处理与Python.ipynb',
 'ch2-获得文本语料和词汇资源.ipynb',
 'ch3-加工原料文本.ipynb',
 'imgs',
 'test.py']

NLTK 中的语料库文件也可以使用这些方法来访问。我们只需使用 `nltk.data.find()`来 获取语料库项目的文件名。然后就可以使用我们刚才讲的方式打开和阅读它。

In [20]:
import nltk
path = nltk.data.find('corpora/gutenberg/melville-moby_dick.txt')

In [22]:
print(str(path))
path

f:\Anaconda3\share\nltk_data\corpora\gutenberg\melville-moby_dick.txt


FileSystemPathPointer('f:\\Anaconda3\\share\\nltk_data\\corpora\\gutenberg\\melville-moby_dick.txt')

In [30]:
raw = open(path, 'rU').read()
# raw
len(raw)
type(raw)

f:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


str

#### 从 PDF、MS Word 及其他二进制格式中提取文本

`ASCII` 码文本和 `HTML` 文本是人可读的格式。文字常常以二进制格式出现，如`PDF`和`MSWord`，只能使用专门的软件打开。第三方函数库如 `pypdf`和`pywin32` 提供了对这些格式的访问。从多列文档中提取文本是特别具有挑战性的。一次性转换几个文件，会比较简单些，用一个合适的应用程序打开文件，以文本格式保存到本地驱动器，然后以如下所述的方式访问它。如果该文档已经在网络上，你可以在Google 的搜索框输入它的URL。搜索结果通常包括这个文档的`HTML`版本的链接，你可以将它保存为文本。

##### 捕获用户输入

有时我们想捕捉用户与我们的程序交互时输入的文本。调用Python 函数`raw_input()` 提示用户输入一行数据。保存用户输入到一个变量后，我们可以像其他字符串那样操纵它。

In [34]:
# s = raw_input("Enter some text:")
s = input("Enter some text:")

In [35]:
print("You typed", len(nltk.word_tokenize(s)), "words.")

You typed 8 words.


##### NLP的流程

![3-1](./imgs/3-1.jpg)

在这条流程后面还有很多操作。要正确理解它，这样有助于明确其中提到的每个变量的
类型。当我们载入一个URL或文件的内容时，或者当我们去掉 HTML 标记时，我们正在处理字符串，也就是Python的`str`数据类型：

In [37]:
raw = open('./test.py').read()
type(raw)

str

当我们将一个字符串分词，会产生一个（词的）链表，这是Python 的`list`类型。规
范化和排序链表产生其它链表：

In [38]:
tokens = nltk.word_tokenize(raw)
type(tokens)

list

In [39]:
words = [w.lower() for w in tokens]
type(words)

list

In [40]:
vocab = sorted(set(words))
type(vocab)

list

一个对象的类型决定了它可以执行哪些操作。比如说我们可以追加元素到一个链表，但不能追加元素到一个字符串：

In [41]:
vocab.append('blog')

In [42]:
raw.append('blog')

AttributeError: 'str' object has no attribute 'append'

In [43]:
# 同样的，我们可以连接字符串与字符串，列出链表内容，但我们不能连接字符串与链表
query = 'Who knows?'
beatles = ['john', 'paul', 'george', 'ringo']
query + beatles

TypeError: can only concatenate str (not "list") to str

### 3.2 字符串：最底层的文本处理

#### 字符串的基本操作

In [1]:
monty = 'Monty Python'
monty

'Monty Python'

In [3]:
circus = 'Monty Python\'s Flying Circus'
circus

"Monty Python's Flying Circus"

In [4]:
# 使用反斜杠或者括号跨行
couplet = "Shall I compare thee to a Summer's day?"\
    "Thou are more lovely and more temperate:"
print(couplet)

Shall I compare thee to a Summer's day?Thou are more lovely and more temperate:


In [6]:
couplet = ("Rough winds do shake the darling buds of May,"
    "And Summer's lease hath all too short a date:")
print(couplet)

Rough winds do shake the darling buds of May,And Summer's lease hath all too short a date:


In [7]:
couplet = """Shall I compare thee to a Summer's day? 
Thou are more lovely and more temperate:"""
print(couplet)

Shall I compare thee to a Summer's day? 
Thou are more lovely and more temperate:


In [8]:
a = [1, 2, 3, 4, 5, 6, 7, 6, 5, 4, 3, 2, 1]
b = [' ' * 2 * (7 - i) + 'very' * i for i in a]
for line in b:
    print(b)

['            very', '          veryvery', '        veryveryvery', '      veryveryveryvery', '    veryveryveryveryvery', '  veryveryveryveryveryvery', 'veryveryveryveryveryveryvery', '  veryveryveryveryveryvery', '    veryveryveryveryvery', '      veryveryveryvery', '        veryveryvery', '          veryvery', '            very']
['            very', '          veryvery', '        veryveryvery', '      veryveryveryvery', '    veryveryveryveryvery', '  veryveryveryveryveryvery', 'veryveryveryveryveryveryvery', '  veryveryveryveryveryvery', '    veryveryveryveryvery', '      veryveryveryvery', '        veryveryvery', '          veryvery', '            very']
['            very', '          veryvery', '        veryveryvery', '      veryveryveryvery', '    veryveryveryveryvery', '  veryveryveryveryveryvery', 'veryveryveryveryveryveryvery', '  veryveryveryveryveryvery', '    veryveryveryveryvery', '      veryveryveryvery', '        veryveryvery', '          veryvery', '            very']
[

#### 更多的字符串操作

![list3-2](./imgs/list3-2.jpg)

In [9]:
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(self, format_spec, /)
 |      Return a formatted version of the string as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  

### 3.3 使用 `Unicode` 进行文字处理

#### 什么是 Unicode？
`Unicode`支持超过一百万种字符。每个字符分配一个编号，称为编码点。在 Python中，
编码点写作`\uXXXX`的形式，其中 `XXXX`是四位十六进制形式数。
文件中的文本都是有特定编码的，所以我们需要一些机制来将文本翻译成`Unicode`——
翻译成 `Unicode` 叫做解码。相对的，要将`Unicode`写入一个文件或终端，我们首先需要将`Unicode` 转化为合适的编码——这种将 `Unicode` 转化为其它编码的过程叫做编码，如图 3-3所示。
![3-3](./imgs/3-3.jpg)
从`Unicode`的角度来看，字符是可以实现一个或多个字形的抽象的实体。只有字形可以出现在屏幕上或被打印在纸上。一个字体是一个字符到字形映射。

#### 从文件中提取已编码文本

假设我们有一个小的文本文件，我们知道它是如何编码的。例如：`polish-lat2.txt` 顾名思义是波兰语的文本片段（来源波兰语Wikipedia；可以在 http://pl.wikipedia.org/wiki/Bibliotek a_Pruska中看到）。此文件是`Latin-2` 编码的，也称为 `ISO-8859-2`。`nltk.data.find()`函数为我们定位文件。

In [14]:
import nltk
path = nltk.data.find('corpora/unicode_samples/polish-lat2.txt')

In [15]:
path

FileSystemPathPointer('f:\\Anaconda3\\share\\nltk_data\\corpora\\unicode_samples\\polish-lat2.txt')

Python的`codecs`模块提供了将编码数据读入为 `Unicode`字符串和将`Unicode` 字符串以编码形式写出的函数。`codecs.open()`函数有一个`encoding`参数来指定被读取或写入的文件的编码。让我们导入`codecs` 模块，以`latin2`为`encoding` 参数，调用它以`Unicode` 打开我们的波兰语文件。

In [24]:
import codecs
f = codecs.open(path, encoding='latin2')

关于`codecs` 允许的 `encoding` 参数列表，可以在 `http://docs.python.org/lib/standard-encodings.html` 中看到。请注意我们可以使用 `f = codecs.open(path, 'w', encoding='utf-8')`写入 `Unicode` 编码数据到一个文件。

从文件对象 `f` 读出的文本将以`Unicode`返回。正如我们较早前指出的，要在终端上查看这个文本，我们需要使用合适的编码对它进行编码。Python特定的编码 `unicode_escape` 是一个虚拟的编码，它把所有非`ASCII` 字符转换成它们的`\uXXXX`形式。编码点在`ASCII`码 `0-127` 的范围以外但低于`256` 的使用两位数字的形式`\xXX`表示。

In [25]:
for line in f:
    line = line.strip()
    # print(line)
    print(line.encode('unicode_escape'))

b'Pruska Biblioteka Pa\\u0144stwowa. Jej dawne zbiory znane pod nazw\\u0105'
b'"Berlinka" to skarb kultury i sztuki niemieckiej. Przewiezione przez'
b'Niemc\\xf3w pod koniec II wojny \\u015bwiatowej na Dolny \\u015al\\u0105sk, zosta\\u0142y'
b'odnalezione po 1945 r. na terytorium Polski. Trafi\\u0142y do Biblioteki'
b'Jagiello\\u0144skiej w Krakowie, obejmuj\\u0105 ponad 500 tys. zabytkowych'
b'archiwali\\xf3w, m.in. manuskrypty Goethego, Mozarta, Beethovena, Bacha.'


在这个输出的第一行有一个以`\u` 转义字符串开始的`Unicode`转义字符串，即`\u0144`。相关的`Unicode`字符将会以字形`ń`显示在屏幕上。在前面例子中的第三行中，我们看到`\xF3`， 对应字形为`ó`，在`128-255` 的范围内。 在Python 中，一个`Unicode`字符串常量可以通过在字符串常量前面加一个 `u`也就是`u'hello'`来指定。任意 `Unicode` 字符通过在`Unicode`字符串常量内使用`\uXXXX`转义序列来定义。我们使**用`ord()`查找一个字符的整数序数**。例如：

In [26]:
ord('a')

97

`97`的十六进制四位数表示是 `0061`，所以我们可以使用相应的转义序列定义一个`Unicode` 字符串常量：

In [27]:
a = u'\u0061'
a

'a'

In [28]:
print(a)

a


请注意，Python的 `print` 语句假设 `Unicode`字符的默认编码是 `ASCII` 码。然而，`ń`不在`ASCII` 码范围之内，所以除非我们指定编码否则不能被输出。在下面的例子中，我们指定 `print` 使用`repr()`转化的字符串，`repr()`输出`UTF-8`转义序列（以`\xXX`的形式），而不是试图显示字形。

如果您的操作系统和区域的设置支持`UTF-8`编码字符，你输入 Python命令：`print(na cute_utf)`，应该能够在你的屏幕上看到`ń`。

In [29]:
nacute = u'\u0144'
nacute

'ń'

In [32]:
nacute_utf = nacute.encode('utf8')
print(repr(nacute))

'ń'


`unicodedata` 模块使我们可以检查`Unicode`字符的属性。在下面的例子中，我们选择 超出`ASCII` 范围的波兰语文本的第三行中的所有字符，输出它们的`UTF-8`转义值，然后是使用标准`Unicode`约定的它们的编码点整数（即以 `U+`为前缀的十六进制数字），随后是它们的`Unicode`名称。

In [33]:
import unicodedata
lines = codecs.open(path, encoding='latin2').readlines()

In [34]:
line = lines[2]
print(line.encode('unicode_escape'))

b'Niemc\\xf3w pod koniec II wojny \\u015bwiatowej na Dolny \\u015al\\u0105sk, zosta\\u0142y\\n'


In [37]:
for c in line:
    if ord(c) > 127:
        print('%r U+%04x %s' % (c.encode('utf8'), ord(c), unicodedata.name(c)))

b'\xc3\xb3' U+00f3 LATIN SMALL LETTER O WITH ACUTE
b'\xc5\x9b' U+015b LATIN SMALL LETTER S WITH ACUTE
b'\xc5\x9a' U+015a LATIN CAPITAL LETTER S WITH ACUTE
b'\xc4\x85' U+0105 LATIN SMALL LETTER A WITH OGONEK
b'\xc5\x82' U+0142 LATIN SMALL LETTER L WITH STROKE


In [38]:
# 下一个例子展示Python 字符串函数和re 模块是如何接收Unicode字符串的
line.find(u'zosta\u0142y')

54

In [39]:
line = line.lower()

In [40]:
print(line.encode('unicode_escape'))

b'niemc\\xf3w pod koniec ii wojny \\u015bwiatowej na dolny \\u015bl\\u0105sk, zosta\\u0142y\\n'


In [41]:
import re
m = re.search(u'\u015b\w*', line)
m.group()

'światowej'

In [42]:
# NLTK 分词器允许Unicode字符串作为输入，并输出相应地Unicode字符串
nltk.word_tokenize(line)

['niemców',
 'pod',
 'koniec',
 'ii',
 'wojny',
 'światowej',
 'na',
 'dolny',
 'śląsk',
 ',',
 'zostały']

#### 在 Python 中使用本地编码

果你习惯了使用特定的本地编码字符，你可能希望能够在一个Python 文件中使用你 的字符串输入及编辑的标准方法。为了做到这一点，你需要在你的文件的第一行或第二行中 包含字符串：`# -*- coding: <coding>-*-` 。请注意，`<coding>`是一个像`latin-1`,
`big5`或者`utf-8`的字符串。


### 3.4 使用正则表达式检测词组搭配

许多语言处理任务都涉及**模式匹配**。例如：我们可以使用`endswith('ed')`找到以`ed` 结尾的词。在表 1-4 中我们可以看到各种“词测试”。正则表达式给我们一个更加强大和灵活的方法描述我们感兴趣的字符模式。

In [44]:
# -*- coding: utf-8 -*-
import re
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]

#### 使用基本的元字符

让我们使用正则表达式`ed$`查找以`ed` 结尾的词汇。函数 `re.search(p, s)`检查字符
串`s` 中是否有模式 `p`。我们需要指定感兴趣的字符，然后使用美元符号，它是正则表达式中有特殊用途的符号，用来匹配单词的末尾：

In [45]:
[w for w in wordlist if re.search('ed$', w)]

['abaissed',
 'abandoned',
 'abased',
 'abashed',
 'abatised',
 'abed',
 'aborted',
 'abridged',
 'abscessed',
 'absconded',
 'absorbed',
 'abstracted',
 'abstricted',
 'accelerated',
 'accepted',
 'accidented',
 'accoladed',
 'accolated',
 'accomplished',
 'accosted',
 'accredited',
 'accursed',
 'accused',
 'accustomed',
 'acetated',
 'acheweed',
 'aciculated',
 'aciliated',
 'acknowledged',
 'acorned',
 'acquainted',
 'acquired',
 'acquisited',
 'acred',
 'aculeated',
 'addebted',
 'added',
 'addicted',
 'addlebrained',
 'addleheaded',
 'addlepated',
 'addorsed',
 'adempted',
 'adfected',
 'adjoined',
 'admired',
 'admitted',
 'adnexed',
 'adopted',
 'adossed',
 'adreamed',
 'adscripted',
 'aduncated',
 'advanced',
 'advised',
 'aeried',
 'aethered',
 'afeared',
 'affected',
 'affectioned',
 'affined',
 'afflicted',
 'affricated',
 'affrighted',
 'affronted',
 'aforenamed',
 'afterfeed',
 'aftershafted',
 'afterthoughted',
 'afterwitted',
 'agazed',
 'aged',
 'agglomerated',
 'aggri

**通配符`.`**

匹配任何单个字符。假设我们有一个8个字母组成的词的字谜室，j 是其第
三个字母，t 是其第六个字母。空白单元格中的每个地方，我们用一个句点：

In [46]:
[w for w in wordlist if re.search('^..j..t..$', w)]

['abjectly',
 'adjuster',
 'dejected',
 'dejectly',
 'injector',
 'majestic',
 'objectee',
 'objector',
 'rejecter',
 'rejector',
 'unjilted',
 'unjolted',
 'unjustly']

插入符号`^`匹配字符串的开始，就像`$`符号匹配字符串的结尾。如果我们不用这两个符号，刚才例子中我们会得到什么样的结果？

In [47]:
[w for w in wordlist if re.search('..j..t..', w)]

['abjectedness',
 'abjection',
 'abjective',
 'abjectly',
 'abjectness',
 'adjection',
 'adjectional',
 'adjectival',
 'adjectivally',
 'adjective',
 'adjectively',
 'adjectivism',
 'adjectivitis',
 'adjustable',
 'adjustably',
 'adjustage',
 'adjustation',
 'adjuster',
 'adjustive',
 'adjustment',
 'antejentacular',
 'antiprojectivity',
 'bijouterie',
 'coadjustment',
 'cojusticiar',
 'conjective',
 'conjecturable',
 'conjecturably',
 'conjectural',
 'conjecturalist',
 'conjecturality',
 'conjecturally',
 'conjecture',
 'conjecturer',
 'coprojector',
 'counterobjection',
 'dejected',
 'dejectedly',
 'dejectedness',
 'dejectile',
 'dejection',
 'dejectly',
 'dejectory',
 'dejecture',
 'disjection',
 'guanajuatite',
 'inadjustability',
 'inadjustable',
 'injectable',
 'injection',
 'injector',
 'injustice',
 'insubjection',
 'interjection',
 'interjectional',
 'interjectionalize',
 'interjectionally',
 'interjectionary',
 'interjectionize',
 'interjectiveness',
 'interjector',
 'interje

最后，**符号`?`表示前面的字符是可选的**。因此`^e-?mail $`将匹配 `email` 和 `e-mail`。我们可以使用`sum(1 for w in text if re.search('^e-? mail$', w))`计数一个文本中这个词（任一拼写形式）出现的总次数。

In [78]:
text = nltk.book.text9
sum(1 for w in text if re.search('^e-?mail $', w))

0

#### 范围与闭包

T9系统用于在手机上输入文本(见图3-5)。两个或两个以上的词汇以相同的击键顺序输入，这叫做输入法联想提示。例如：`hole` 和 `golf` 都是通过输入序列`4653`。还有哪些其它词汇由相同的序列产生？这里我们使用正则表达式`^[ghi][mno][jlk][def]$`：
![3-5](./imgs/3-5.jpg)

In [79]:
[w for w in wordlist if re.search('^[ghi][mno][jlk][def]$', w)]

['gold', 'golf', 'hold', 'hole']

来看一些“手指绕口令”，只用一部分数字键盘搜索词汇。例如： `^[ghijklmno]+$`，或更为简洁的：`^[g-o]+$`，将匹配只使用中间行的4、5、6键的词汇，`^[a-fj-o]+$`将匹配使用右上角2、3、5、6 键的词汇。`-`和`+`是什么意思？

让我们进一步探索`+`符号。请注意，它可以适用于单个字母或括号内的字母集：

In [80]:
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))
[w for w in chat_words if re.search('^m+i+n+e+$', w)]

['miiiiiiiiiiiiinnnnnnnnnnneeeeeeeeee',
 'miiiiiinnnnnnnnnneeeeeeee',
 'mine',
 'mmmmmmmmiiiiiiiiinnnnnnnnneeeeeeee']

In [81]:
[w for w in chat_words if re.search('^[ha]+$',w)]

['a',
 'aaaaaaaaaaaaaaaaa',
 'aaahhhh',
 'ah',
 'ahah',
 'ahahah',
 'ahh',
 'ahhahahaha',
 'ahhh',
 'ahhhh',
 'ahhhhhh',
 'ahhhhhhhhhhhhhh',
 'h',
 'ha',
 'haaa',
 'hah',
 'haha',
 'hahaaa',
 'hahah',
 'hahaha',
 'hahahaa',
 'hahahah',
 'hahahaha',
 'hahahahaaa',
 'hahahahahaha',
 'hahahahahahaha',
 'hahahahahahahahahahahahahahahaha',
 'hahahhahah',
 'hahhahahaha']

很显然，`+`表示的是**前面的项目的一个或多个实例**，它可以是单独的字母如`m`，
可以是一个集合如`[fed]`，可以是一个范围如`[d-f]`。现在让我们用`*`替换`+`，它表示**前面的项目的零个或多个实例**。正则表达式`^m*i*n*e*$`将匹配所有我们用`^m+i +n+e+$`找到的，同时包括其中一些字母不出现的词汇，例如：`me`、`min` 和`mmmmm`。 请注意`+`和`*`符号有时被称为的 `Kleene` 闭包，或者**闭包**。

运算符`^`当它出现在方括号内的第一个字符位置时有另外的功能。例如：`[^aeiouAEIOU]`匹配除元音字母之外的所有字母。我们可以搜索`NPS`聊天语料库中完全由非元音字母组成的词汇，使用`^[^aeiouAEIOU]+$`查找诸如`:):):)`、`grrr`、`cyb3r` 和`zzzzzzzz` 这样的词。请注意其中包含非字母字符。

下面是另外一些正则表达式的例子，用来寻找匹配特定模式的词汇标识符，这些例子演示如何使用一些新的符号：`\`，`{}`，`()`和`|`。

In [83]:
[w for w in chat_words if re.search('^[^aeiouAEIOU]+$',w)]

['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '!!!!!!',
 '!!!!!!!',
 '!!!!!!!!',
 '!!!!!!!!!',
 '!!!!!!!!!!',
 '!!!!!!!!!!!',
 '!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!.',
 '!!!!!.',
 '!!!!....',
 '!!!.',
 '!!.',
 '!!...',
 '!.',
 '!...',
 '!=',
 '!?',
 '!??',
 '!???',
 '"',
 '"...',
 '"?',
 '"s',
 '#',
 '###',
 '####',
 '$',
 '$$',
 '$27',
 '&',
 '&^',
 "'",
 "''",
 "'.",
 "'d",
 "'ll",
 "'m",
 "'n'",
 "'s",
 '(',
 '(((',
 '((((',
 '(((((',
 '((((((',
 '(((((((',
 '((((((((',
 '(((((((((',
 '((((((((((',
 '(((((((((((',
 '((((((((((((',
 '(((((((((((((',
 '((((((((((((((',
 '(((((((((((((((',
 '(((((((((((((((((',
 '((((((((((((((((((',
 '((((((((((((((((((((',
 '(((((((((((((((((((((',
 '(((((((((((((((((((((((',
 '((((((((((((((((((((((((',
 '(((((((((((((((((((((((((',
 '((((((((((((((((((((((((((',
 '((((

In [84]:
wsj = sorted(set(nltk.corpus.treebank.words()))
[w for w in wsj if re.search('^[0-9]+\.[0-9]+$', w)]

['0.0085',
 '0.05',
 '0.1',
 '0.16',
 '0.2',
 '0.25',
 '0.28',
 '0.3',
 '0.4',
 '0.5',
 '0.50',
 '0.54',
 '0.56',
 '0.60',
 '0.7',
 '0.82',
 '0.84',
 '0.9',
 '0.95',
 '0.99',
 '1.01',
 '1.1',
 '1.125',
 '1.14',
 '1.1650',
 '1.17',
 '1.18',
 '1.19',
 '1.2',
 '1.20',
 '1.24',
 '1.25',
 '1.26',
 '1.28',
 '1.35',
 '1.39',
 '1.4',
 '1.457',
 '1.46',
 '1.49',
 '1.5',
 '1.50',
 '1.55',
 '1.56',
 '1.5755',
 '1.5805',
 '1.6',
 '1.61',
 '1.637',
 '1.64',
 '1.65',
 '1.7',
 '1.75',
 '1.76',
 '1.8',
 '1.82',
 '1.8415',
 '1.85',
 '1.8500',
 '1.9',
 '1.916',
 '1.92',
 '10.19',
 '10.2',
 '10.5',
 '107.03',
 '107.9',
 '109.73',
 '11.10',
 '11.5',
 '11.57',
 '11.6',
 '11.72',
 '11.95',
 '112.9',
 '113.2',
 '116.3',
 '116.4',
 '116.7',
 '116.9',
 '118.6',
 '12.09',
 '12.5',
 '12.52',
 '12.68',
 '12.7',
 '12.82',
 '12.97',
 '120.7',
 '1206.26',
 '121.6',
 '126.1',
 '126.15',
 '127.03',
 '129.91',
 '13.1',
 '13.15',
 '13.5',
 '13.50',
 '13.625',
 '13.65',
 '13.73',
 '13.8',
 '13.90',
 '130.6',
 '130.7',
 '

In [85]:
[w for w in wsj if re.search('^[A-Z]+\$$', w)]

['C$', 'US$']

In [86]:
[w for w in wsj if re.search('^[0-9]{4}$', w)] # {n}表示出现n次

['1614',
 '1637',
 '1787',
 '1901',
 '1903',
 '1917',
 '1925',
 '1929',
 '1933',
 '1934',
 '1948',
 '1953',
 '1955',
 '1956',
 '1961',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1975',
 '1976',
 '1977',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2005',
 '2009',
 '2017',
 '2019',
 '2029',
 '3057',
 '8300']

In [87]:
[w for w in wsj if re.search('^[0-9]+-[a-z]{3,5}$', w)] # {3,5}出现3或5次

['10-day',
 '10-lap',
 '10-year',
 '100-share',
 '12-point',
 '12-year',
 '14-hour',
 '15-day',
 '150-point',
 '190-point',
 '20-point',
 '20-stock',
 '21-month',
 '237-seat',
 '240-page',
 '27-year',
 '30-day',
 '30-point',
 '30-share',
 '30-year',
 '300-day',
 '36-day',
 '36-store',
 '42-year',
 '50-state',
 '500-stock',
 '52-week',
 '69-point',
 '84-month',
 '87-store',
 '90-day']

In [89]:
[w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}$', w)]

['black-and-white',
 'bread-and-butter',
 'father-in-law',
 'machine-gun-toting',
 'savings-and-loan']

In [90]:
[w for w in wsj if re.search('(ed|ing)$', w)]

['62%-owned',
 'Absorbed',
 'According',
 'Adopting',
 'Advanced',
 'Advancing',
 'Alfred',
 'Allied',
 'Annualized',
 'Anything',
 'Arbitrage-related',
 'Arbitraging',
 'Asked',
 'Assuming',
 'Atlanta-based',
 'Baking',
 'Banking',
 'Beginning',
 'Beijing',
 'Being',
 'Bermuda-based',
 'Betting',
 'Boeing',
 'Broadcasting',
 'Bucking',
 'Buying',
 'Calif.-based',
 'Change-ringing',
 'Citing',
 'Concerned',
 'Confronted',
 'Conn.based',
 'Consolidated',
 'Continued',
 'Continuing',
 'Declining',
 'Defending',
 'Depending',
 'Designated',
 'Determining',
 'Developed',
 'Died',
 'During',
 'Encouraged',
 'Encouraging',
 'English-speaking',
 'Estimated',
 'Everything',
 'Excluding',
 'Exxon-owned',
 'Faulding',
 'Fed',
 'Feeding',
 'Filling',
 'Filmed',
 'Financing',
 'Following',
 'Founded',
 'Fracturing',
 'Francisco-based',
 'Fred',
 'Funded',
 'Funding',
 'Generalized',
 'Germany-based',
 'Getting',
 'Guaranteed',
 'Having',
 'Heating',
 'Heightened',
 'Holding',
 'Housing',
 'Illumin

你可能已经知道反斜杠表示其后面的字母不再有特殊的含义而是按照字面的表示匹配词中特定的字符。因此，虽然`.`有特殊含义，但`\.`值匹配一个句号。大括号表达式，如`{3,5}`，表示前面的**项目重复指定次数**。`管道字符|`表示从其左边的内容和右边的内容中选择一个。圆括号表示一个操作符的范围，它们可以与管道（或叫`析取`）符号一起使用，如： `w(i|e|ai|oo)t`，匹配`wit`、`wet`、`wait` 和`woot`。你可以省略这个例子里的最后一个表达式中的括号，使用`ed|ing$`搜索看看会发生什么，这是很有益处的。

In [92]:
[w for w in wsj if re.search('ed|ing$', w)] ## en/nn出现了

['62%-owned',
 'Absorbed',
 'According',
 'Adopting',
 'Advanced',
 'Advancing',
 'Alfred',
 'Allied',
 'Annualized',
 'Anything',
 'Arbitrage-related',
 'Arbitraging',
 'Asked',
 'Assuming',
 'Atlanta-based',
 'Baking',
 'Banking',
 'Beginning',
 'Beijing',
 'Being',
 'Bermuda-based',
 'Betting',
 'Biedermann',
 'Boeing',
 'Breeden',
 'Broadcasting',
 'Bucking',
 'Buying',
 'Calif.-based',
 'Cathedral',
 'Cedric',
 'Change-ringing',
 'Citing',
 'Concerned',
 'Confederation',
 'Confronted',
 'Conn.based',
 'Consolidated',
 'Continued',
 'Continuing',
 'Credit',
 'Declining',
 'Defending',
 'Depending',
 'Designated',
 'Determining',
 'Developed',
 'Died',
 'During',
 'Encouraged',
 'Encouraging',
 'English-speaking',
 'Estimated',
 'Everything',
 'Excluding',
 'Exxon-owned',
 'Faulding',
 'Fed',
 'Federal',
 'Federalist',
 'Federation',
 'Feeding',
 'Filling',
 'Filmed',
 'Financing',
 'Following',
 'Founded',
 'Fracturing',
 'Francisco-based',
 'Fred',
 'Freddie',
 'Frederick',
 'Frie

![re1](./imgs/re1.jpg)
![re2](./imgs/re2.jpg)

对Python 解释器而言，一个正则表达式与任何其他字符串没有两样。如果字符串中包含一个反斜杠后面跟一些特殊字符，Python解释器将会特殊处理它们。例如：`\b`会被解释为一个退格符号。一般情况下，当使用含有反斜杠的正则表达式时，我们应该告诉解释器一定不要解释字符串里面的符号，而仅仅是将它直接传递给`re`库来处理。我们通过给字符串加一个前缀`r`来表明它是一个原始字符串。例如：原始字符串`r'\band\b'`包含两个`\b`符号会被 `re` 库解释为匹配词的边界而不是解释为退格字符。如果你能逐渐习惯使用 `r'... '`表示正则表达式——就像从现在开始我们将做的那样——你将会避免去想这些解释上的歧义。


### 3.5 正则表达式的有益应用


前面的例子都涉及到使用`re.search(regexp, w)`匹配一些正则表达式 `regexp` 来搜索
词`w`。除了检查一个正则表达式是否匹配一个单词外，我们还可以使用正则表达式从词汇中提取的特征或以特殊的方式来修改词。


#### 提取字符块

通过`re.findall()` (“find all”即找到所有)方法找出所有（无重叠的）匹配指定正则表达式的。让我们找出一个词中的元音，再计数它们:

In [2]:
import re
word = 'supercalifragilisticexpialidocious'
re.findall(r'[aeiou]', word)

['u',
 'e',
 'a',
 'i',
 'a',
 'i',
 'i',
 'i',
 'e',
 'i',
 'a',
 'i',
 'o',
 'i',
 'o',
 'u']

In [3]:
len(re.findall(r'[aeiou]', word))

16

让我们来看看一些文本中的**两个或两个以上**的元音序列，并确定它们的相对频率：

In [4]:
import nltk
wsj = sorted(set(nltk.corpus.treebank.words()))
fd = nltk.FreqDist(vs for word in wsj
                        for vs in re.findall(r'[aeiou]{2,}', word))

fd.items()                    

dict_items([('ea', 476), ('oi', 65), ('ou', 329), ('io', 549), ('ee', 217), ('ie', 331), ('ui', 95), ('ua', 109), ('ai', 261), ('ue', 105), ('ia', 253), ('ei', 86), ('iai', 1), ('oo', 174), ('au', 106), ('eau', 10), ('oa', 59), ('oei', 1), ('oe', 15), ('eo', 39), ('uu', 1), ('eu', 18), ('iu', 14), ('aii', 1), ('aiia', 1), ('ae', 11), ('aa', 3), ('oui', 6), ('ieu', 3), ('ao', 6), ('iou', 27), ('uee', 4), ('eou', 5), ('aia', 1), ('uie', 3), ('iao', 1), ('eei', 2), ('uo', 8), ('uou', 5), ('eea', 1), ('ueui', 1), ('ioa', 1), ('ooi', 1)])

In [9]:
[int(n) for n in re.findall(r'[0-9]+', '2009-12-31')]

[2009, 12, 31]

#### 在字符块上做更多事情

一旦我们会使用`re.findall()`从词中提取字符块，就可以在这些字符块上做一些有趣的 事情，**例如将它们粘贴在一起或用它们绘图**。

英文文本是高度冗余的，忽略掉词内部的元音仍然可以很容易的阅读，有些时候这很明
显。例如：`declaration` 变成 `dclrtn`，`inalienable` 变成 `inlnble`，保留所有词首或词尾的元音序列。 在下一个例子中，正则表达式匹配词首元音序列，词尾元音序列和所有的辅音；其它的被忽略。这三个阶段从左到右处理，如果词匹配了三个部分之一，正则表达式后面的部分将被忽略。我们使用`re.findall()`提取所有匹配的词中的字符，然后使用`''.join()`将它们连接在一起。

In [10]:
regexp = r'^[AEIOUaeiou]+|[AEIOUaeiou]+$|[^AEIOUaeiou]'
def compress(word):
    pieces = re.findall(regexp, word)
    return ''.join(pieces)

english_udhr = nltk.corpus.udhr.words('English-Latin1')
print(nltk.tokenwrap(compress(w) for w in english_udhr[:75]))

Unvrsl Dclrtn of Hmn Rghts Prmble Whrs rcgntn of the inhrnt dgnty and
of the eql and inlnble rghts of all mmbrs of the hmn fmly is the fndtn
of frdm , jstce and pce in the wrld , Whrs dsrgrd and cntmpt fr hmn
rghts hve rsltd in brbrs acts whch hve outrgd the cnscnce of mnknd ,
and the advnt of a wrld in whch hmn bngs shll enjy frdm of spch and


接下来，让我们将正则表达式与条件频率分布结合起来。在这里，我们将从罗托卡特语词汇中提取所有辅音-元音序列，如`ka`和 `si`。因为每部分都是成对的，它可以被用来初始化一个条件频率分布。然后我们为每对的频率列表。

In [11]:
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')

In [12]:
cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]

In [13]:
cfd = nltk.ConditionalFreqDist(cvs)

In [14]:
cfd.tabulate()

    a   e   i   o   u 
k 418 148  94 420 173 
p  83  31 105  34  51 
r 187  63  84  89  79 
s   0   0 100   2   1 
t  47   8   0 148  37 
v  93  27 105  48  49 


如果我们想要检查表格中数字背后的词汇，有一个索引允许我们迅速找到包含一个给定
的辅音-元音对的单词的列表将会有帮助。例如：`cv_index['su']`应该给我们所有含有“su” 的词汇。下面是我们如何能做到这一点：

In [15]:
cv_word_pairs = [(cv, w) for w in rotokas_words
                         for cv in re.findall(r'[ptksvr][aeiou]', w)]

cv_index = nltk.Index(cv_word_pairs)

In [16]:
cv_index['su']

['kasuari']

In [17]:
cv_index['po']

['kaapo',
 'kaapopato',
 'kaipori',
 'kaiporipie',
 'kaiporivira',
 'kapo',
 'kapoa',
 'kapokao',
 'kapokapo',
 'kapokapo',
 'kapokapoa',
 'kapokapoa',
 'kapokapora',
 'kapokapora',
 'kapokaporo',
 'kapokaporo',
 'kapokari',
 'kapokarito',
 'kapokoa',
 'kapoo',
 'kapooto',
 'kapoovira',
 'kapopaa',
 'kaporo',
 'kaporo',
 'kaporopa',
 'kaporoto',
 'kapoto',
 'karokaropo',
 'karopo',
 'kepo',
 'kepoi',
 'keposi',
 'kepoto']

In [18]:
cv_index

Index(list,
      {'ka': ['kaa',
        'kaa',
        'kaa',
        'kaakaaro',
        'kaakaaro',
        'kaakaaviko',
        'kaakaaviko',
        'kaakaavo',
        'kaakaavo',
        'kaakaoko',
        'kaakaoko',
        'kaakasi',
        'kaakasi',
        'kaakau',
        'kaakau',
        'kaakauko',
        'kaakauko',
        'kaakito',
        'kaakuupato',
        'kaaova',
        'kaapa',
        'kaapea',
        'kaapie',
        'kaapie',
        'kaapiepato',
        'kaapisi',
        'kaapisivira',
        'kaapo',
        'kaapopato',
        'kaara',
        'kaare',
        'kaareko',
        'kaarekopie',
        'kaareto',
        'kaava',
        'kaavaaua',
        'kaaveaka',
        'kaaveaka',
        'kaaveakapie',
        'kaaveakapie',
        'kaaveakapievira',
        'kaaveakapievira',
        'kaaveakavira',
        'kaaveakavira',
        'kae',
        'kae',
        'kaekae',
        'kaekae',
        'kaekae',
        'kaekae',
      

#### 查找词干
对于一些语言处理任务，我们想忽略词语结尾，只是处理词干。抽出一个词的词干的方法有很多种。这里的是一种简单直观的方法，直接去掉任何看起来像一个后缀的字符:

In [1]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

虽然我们最终将使用NLTK中内置的词干，看看我们如何能够使用正则表达式处理这个任务是有趣的。我们的第一步是建立一个所有后缀的连接。我们需要把它放在括号内以限制这个连接的范围。

In [3]:
import re
re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

['ing']

在这里，尽管正则表达式匹配整个单词，`re.findall()`只是给我们后缀。这是因为括号
有第二个功能：选择要提取的子字符串。如果我们要使用括号来指定连接的范围，但不想选 择要输出的字符串，必须添加`?:`，它是许多神秘奥妙的正则表达式之一。下面是改进后 的版本：

In [4]:
re.findall(r'^.*(?:ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

['processing']

然而，实际上，我们会想将词分成词干和后缀。所以，我们应该只是用括号括起正则表达式的这两个部分：

In [5]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

[('process', 'ing')]

`*`操作符是“贪婪的”，所以表达式的`.*`部分试图尽可能多的匹配输入的字符串。如果我 们使用“非贪婪”版本的`*`操作符，写成`*?`，我们就得到我们想要的：

In [6]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

[('process', 'es')]

我们甚至可以通过使第二个括号中的内容变成可选，来得到空后缀:

In [7]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$', 'language')

[('language', '')]

这种方法仍然有许多问题，（你能发现它们吗？）但我们仍将继续定义一个函数来获取 词干，并将它应用到整个文本：

In [8]:
def stem(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

In [10]:
import nltk
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government. Supreme executive power derives from 
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = nltk.word_tokenize(raw)
[stem(t) for t in tokens]

['DENNIS',
 ':',
 'Listen',
 ',',
 'strange',
 'women',
 'ly',
 'in',
 'pond',
 'distribut',
 'sword',
 'i',
 'no',
 'basi',
 'for',
 'a',
 'system',
 'of',
 'govern',
 '.',
 'Supreme',
 'execut',
 'power',
 'deriv',
 'from',
 'a',
 'mandate',
 'from',
 'the',
 'mass',
 ',',
 'not',
 'from',
 'some',
 'farcical',
 'aquatic',
 'ceremony',
 '.']

#### 搜索已分词文本

你可以使用一种特殊的正则表达式搜索一个文本中多个词（这里的文本是一个标识符列
表）。例如：`<a> <man>`找出文本中所有`a man`的实例。**尖括号**用于标记标识符的边界，尖括号之间的所有空白都被忽略（这只对 NLTK中的 `findall()`方法处理文本有效）。 在下面的例子中，我们使用`<.*>`，它**将匹配所有单个标识符**，将它括在括号里，于是只匹配词（例如：`monied`）而不匹配短语（例如：`a monied man`）。第二个例子找出以词`bro`结尾的三个词组成的短语。最后一个例子找出以字母`l`开始的**三个或更多**词组成的序列。

In [11]:
from nltk.corpus import gutenberg, nps_chat
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))
moby.findall(r"<a> (<.*>) <man>")

monied; nervous; dangerous; white; white; white; pious; queer; good;
mature; white; Cape; great; wise; wise; butterless; white; fiendish;
pale; furious; better; certain; complete; dismasted; younger; brave;
brave; brave; brave


In [12]:
chat = nltk.Text(nps_chat.words())
chat.findall(r"<.*><.*> <bro>")

you rule bro; telling you bro; u twizted bro


In [13]:
chat.findall(r"<l.*>{3,}")

lol lol lol; lmao lol lol; lol lol lol; la la la la la; la la la; la
la la; lovely lol lol love; lol lol lol.; la la la; la la la


In [15]:
#它能标注字符串 s中所有匹配模式 p的地方
nltk.re_show(r'(aeiou)$', 'sasodasdasad')

sasodasdasad


In [16]:
# 探索正则表达式的图形界面
nltk.app.nemo()

当我们研究的语言现象与特定词语相关时建立搜索模式是很容易的。在某些情况下，一个小小的创意可能会花很大功夫。例如：在大型文本语料库中搜索`x and other ys`形式的表达式能让我们发现上位词（见 2.5节）:

In [17]:
from nltk.corpus import brown
hobbies_learned = nltk.Text(brown.words(categories=['hobbies', 'learned']))
hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")

speed and other activities; water and other liquids; tomb and other
landmarks; Statues and other monuments; pearls and other jewels;
charts and other items; roads and other features; figures and other
objects; military and other areas; demands and other factors;
abstracts and other compilations; iron and other metals


In [18]:
# 查找模式“as x as y”的实例以发现实体及其属性信息
hobbies_learned.findall(r"<as> <\w*> <as> <\w*>")

as accurately as possible; as well as the; as faithfully as possible;
as much as what; as neat as a; as simple as you; as well as other; as
well as other; as involved as determining; as well as other; as
important as another; as accurately as possible; as accurate as any;
as much as any; as different as a; as Orphic as that; as coppery as
Delawares; as good as another; as large as small; as well as ease; as
well as their; as well as possible; as straight as possible; as well
as nailed; as smoothly as the; as soon as a; as well as injuries; as
well as many; as well as reason; as well as in; as well as of; as well
as a; as well as summer; as well as providing; as important as
cooling; as evenly as it; as much as shading; as well as some; as well
as subsoil; as high as possible; as well as many; as general as
electrical; as long as the; as well as the; as much as was; as well as
set; as well as by; as high as 15; as well as aid; as much as
possible; as well as personalities; as low as a; 

### 3.6 规范化文本

使用 `lower()`我们将文本规范化为小写，这样一来“The”与 “the”的区别被忽略。我们常常想比这走得更远，例如：去掉所有的词缀以及提取词干的任务等。更进一步的步骤是确保结果形式是字典中确定的词，即叫做**词形归并**的任务。我们依次讨论这些。首先，我们需要定义我们将在本节中使用的数据：

In [21]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords 
is no basis for a system of government. Supreme executive power derives from 
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = nltk.word_tokenize(raw)

#### 词干提取器

NLTK 中包括了一些现成的词干提取器，如果你需要一个词干提取器，你应该优先使用它们中的一个，而不是使用正则表达式制作自己的词干提取器，因为NLTK 中的词干提取器能处理的不规则的情况很广泛。`Porter` 和`Lancaster` 词干提取器按照它们自己的规则剥离词缀。请看`Porter`词干提取器正确处理了词 `lying`（将它映射为 `lie``），而Lancaster` 词干提取器并没有处理好。

In [20]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
[porter.stem(t) for t in tokens]

['denni',
 ':',
 'listen',
 ',',
 'strang',
 'women',
 'lie',
 'in',
 'pond',
 'distribut',
 'sword',
 'is',
 'no',
 'basi',
 'for',
 'a',
 'system',
 'of',
 'govern',
 '.',
 'suprem',
 'execut',
 'power',
 'deriv',
 'from',
 'a',
 'mandat',
 'from',
 'the',
 'mass',
 ',',
 'not',
 'from',
 'some',
 'farcic',
 'aquat',
 'ceremoni',
 '.']

In [21]:
[lancaster.stem(t) for t in tokens]

['den',
 ':',
 'list',
 ',',
 'strange',
 'wom',
 'lying',
 'in',
 'pond',
 'distribut',
 'sword',
 'is',
 'no',
 'bas',
 'for',
 'a',
 'system',
 'of',
 'govern',
 '.',
 'suprem',
 'execut',
 'pow',
 'der',
 'from',
 'a',
 'mand',
 'from',
 'the',
 'mass',
 ',',
 'not',
 'from',
 'som',
 'farc',
 'aqu',
 'ceremony',
 '.']

词干提取过程没有明确定义，我们通常选择心目中最适合我们的应用的词干提取器。如果你要索引一些文本和使搜索支持不同词汇形式的话，`Porter` 词干提取器是一个很好的选择 

In [16]:
# 例3-1. 使用词干提取器索引文本
import nltk
class IndexedText(object):
    def __init__(self, stemmer, text) -> None:
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
            for (i, word) in enumerate(text))
    
    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = width // 4 #这里使用整除得到int类型
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '%*s' % (width, lcontext[-width:])
            rdisplay = '%-*s' % (width, rcontext[:width])
            print(ldisplay, rdisplay)
    
    def _stem(self, word):
        return self._stemmer.stem(word).lower()

In [17]:
porter = nltk.PorterStemmer()
grail = nltk.corpus.webtext.words('grail.txt')
text = IndexedText(porter, grail)

In [18]:
text.concordance('lie')

r king ! DENNIS : Listen , strange women lying in ponds distributing swords is no
 beat a very brave retreat . ROBIN : All lies ! MINSTREL : [ singing ] Bravest of
       Nay . Nay . Come . Come . You may lie here . Oh , but you are wounded !   
doctors immediately ! No , no , please ! Lie down . [ clap clap ] PIGLET : Well  
ere is much danger , for beyond the cave lies the Gorge of Eternal Peril , which 
   you . Oh ... TIM : To the north there lies a cave -- the cave of Caerbannog --
h it and lived ! Bones of full fifty men lie strewn about its lair . So , brave k
not stop our fight ' til each one of you lies dead , and the Holy Grail returns t


#### 词形归并

`WordNet` 词形归并器删除词缀产生的词都是在它的字典中的词。这个额外的检查过程使 词形归并器比刚才提到的词干提取器要慢。请注意，它并没有处理`lying`，但它将`women`转换为`woman`。

如果你想编译一些文本的词汇，或者想要一个有效词条（或中心词）列表，`WordNet` 词形归并器是一个不错的选择。

In [19]:
wnl = nltk.WordNetLemmatizer()

In [22]:
[wnl.lemmatize(t) for t in tokens]

['DENNIS',
 ':',
 'Listen',
 ',',
 'strange',
 'woman',
 'lying',
 'in',
 'pond',
 'distributing',
 'sword',
 'is',
 'no',
 'basis',
 'for',
 'a',
 'system',
 'of',
 'government',
 '.',
 'Supreme',
 'executive',
 'power',
 'derives',
 'from',
 'a',
 'mandate',
 'from',
 'the',
 'mass',
 ',',
 'not',
 'from',
 'some',
 'farcical',
 'aquatic',
 'ceremony',
 '.']

### 3.7 用正则表达式为文本分词

分词是**将字符串切割成可识别的构成一块语言数据的语言单元**。虽然这是一项基础任 务，我们能够一直拖延到现在为止才讲，是因为许多语料库已经分过词了，也因为NLTK
中包括一些分词器。现在你已经熟悉了正则表达式，你可以学习如何使用它们来为文本分词， 并对此过程中有更多的掌控权。

#### 分词的简单方法

文本分词的一种非常简单的方法是在空格符处分割文本。考虑以下摘自《爱丽丝梦游仙境》中的文本：

In [23]:
raw = """'When I'M a Duchess,' she said to herself, (not in a very hopeful tone 
though), 'I won't have any pepper in my kitchen AT ALL. Soup does very 
well without--Maybe it's always pepper that makes people hot-tempered,'
"""

我们可以使用`raw.split()`在空格符处分割原始文本。使用正则表达式能做同样的事情， 匹配字符串中的所有空格符是不够的，因为这将导致分词结果包含`\n`换行符；我们**需要匹配任何数量的空格符、制表符或换行符**：

In [24]:
import re
re.split(r' ', raw)

["'When",
 "I'M",
 'a',
 "Duchess,'",
 'she',
 'said',
 'to',
 'herself,',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 '\nthough),',
 "'I",
 "won't",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL.',
 'Soup',
 'does',
 'very',
 '\nwell',
 'without--Maybe',
 "it's",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 "hot-tempered,'\n"]

In [25]:
re.split(r'[ \t\n]+', raw)

["'When",
 "I'M",
 'a',
 "Duchess,'",
 'she',
 'said',
 'to',
 'herself,',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 'though),',
 "'I",
 "won't",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL.',
 'Soup',
 'does',
 'very',
 'well',
 'without--Maybe',
 "it's",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 "hot-tempered,'",
 '']

正则表达式`[ \t\n]+`匹配**一个或多个空格、制表符（`\t`）或换行符（`\n`）**。其他空白字符，如回车和换页符，确实应该包含的太多。于是，我们将使用一个 `re` 库内置的缩写`\s`，它表示**匹配所有空白字符**。前面的例子中第二条语句可以改写为`re.split(r'\s+', raw)`。

> 记住在正则表达式前加字母`r`，它告诉 Python解释器按照字面表示对待字符串而不去处理正则表达式中包含的反斜杠字符

In [26]:
re.split(r'\s+', raw)

["'When",
 "I'M",
 'a',
 "Duchess,'",
 'she',
 'said',
 'to',
 'herself,',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 'though),',
 "'I",
 "won't",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL.',
 'Soup',
 'does',
 'very',
 'well',
 'without--Maybe',
 "it's",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 "hot-tempered,'",
 '']

在空格符处分割文本给我们如`(not`和`herself`,”这样的标识符。另一种方法是使
用Python提供给我们的字符类 **`\w`匹配词中的字符**，相当于`[a-zA-Z0-9_]`。也定义了这个类的**补** `\W`即所有字母、数字和下划线**以外**的字符。我们可以在一个简单的正则表达式中用`\W`来分割所有单词字符以外的输入。

In [27]:
re.split(r'\W+', raw)

['',
 'When',
 'I',
 'M',
 'a',
 'Duchess',
 'she',
 'said',
 'to',
 'herself',
 'not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 'though',
 'I',
 'won',
 't',
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL',
 'Soup',
 'does',
 'very',
 'well',
 'without',
 'Maybe',
 'it',
 's',
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 'hot',
 'tempered',
 '']

可以看到，在开始和结尾都给了我们一个空字符串（要了解原因请尝试`'xx'.split('x')`）。通过`re.findall(r'\w+', raw)`使用模式**匹配词汇**而不是空白符号，我们得到相同的标识符， 但没有空字符串。现在，我们正在匹配词汇，我们处在扩展正则表达式覆盖更广泛的情况的位置。正则表达式`\w+|\S\w*`将首先尝试匹配词中字符的所有序列。如果没有找到匹配的，它会尝试匹配后面跟着词中字符的任何非空白字符(`\S`是`\s`的补）。这意味着标点会与跟在后面的字母（如`'s`）在一起，但两个或两个以上的标点字符序列会被分割。

In [28]:
'xx'.split('x')

['', '', '']

In [30]:
re.findall(r'\w+|\S\w*', raw)

["'When",
 'I',
 "'M",
 'a',
 'Duchess',
 ',',
 "'",
 'she',
 'said',
 'to',
 'herself',
 ',',
 '(not',
 'in',
 'a',
 'very',
 'hopeful',
 'tone',
 'though',
 ')',
 ',',
 "'I",
 'won',
 "'t",
 'have',
 'any',
 'pepper',
 'in',
 'my',
 'kitchen',
 'AT',
 'ALL',
 '.',
 'Soup',
 'does',
 'very',
 'well',
 'without',
 '-',
 '-Maybe',
 'it',
 "'s",
 'always',
 'pepper',
 'that',
 'makes',
 'people',
 'hot',
 '-tempered',
 ',',
 "'"]

让我们扩展前面表达式中的`\w+`，允许**连字符和撇号**：`\w+([-']\w+)*`。这个
表达式表示`\w+`后面跟**零个或更多**`[-']\w+`的实例；它会匹配 `hot-tempered`和 `it's`。 （我们需要在这个表达式中包含`?:`，原因前面已经讨论过。）我们还将添加一个模式来匹配引号字符让它们与它们包括的文字分开。

In [33]:
print(re.findall(r"\w+(?:[-']\w+)*|'|[-.()]+|\S\w*", raw))

["'", 'When', "I'M", 'a', 'Duchess', ',', "'", 'she', 'said', 'to', 'herself', ',', '(', 'not', 'in', 'a', 'very', 'hopeful', 'tone', 'though', ')', ',', "'", 'I', "won't", 'have', 'any', 'pepper', 'in', 'my', 'kitchen', 'AT', 'ALL', '.', 'Soup', 'does', 'very', 'well', 'without', '--', 'Maybe', "it's", 'always', 'pepper', 'that', 'makes', 'people', 'hot-tempered', ',', "'"]


In [41]:
text = 'That U.S.A. poster-print costs $12.40...'
# pattern = r'''''(?x) # set flag to allow verbose regexps
# ([A-Z]\.)+ # abbreviations, e.g. U.S.A.
# | \w+(-\w+)* # words with optional internal hyphens
# | \$?\d+(\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
# | \.\.\. # ellipsis
# | [][.,;"'?():-_`] # these are separate tokens; includes ], [
# '''
pattern = r"""(?x)          # set flag to allow verbose regexps 
       (?:[A-Z]\.)+      # abbreviations, e.g. U.S.A. 
       |\d+(?:\.\d+)?%?    # numbers, incl. currency and percentages 
       |\w+(?:[-']\w+)*    # words w/ optional internal hyphens/apostrophe 
       |\.\.\.        # ellipsis 
       |(?:[.,;"'?():-_`])  # special characters with meanings 
      """
nltk.regexp_tokenize(text, pattern)

['That', 'U.S.A.', 'poster-print', 'costs', '12.40', '...']

我们可以使用`set(tokens).difference(wordlist)`，通过比较分词结果与一个词表，然后报告任何没有在词表出现的标识符，来评估一个分词器。 你可能想先将所有标记变成小写。

#### 分词的进一步问题

分词的最后一个问题是缩写的存在，如`didn't`。如果我们想分析一个句子的意思，将这种形式规范化为两个独立的形式：`did`和`n't`(不是 `not`)可能更加有用。我们可以通过 查表来做这项工作。


### 3.8 分割

分词是一个更普遍的分割问题的一个实例。在本节中，我们将看到这个问题的另外两个实例，它们使用与到目前为止我们已经在本章看到的完全不同的技术。

#### 断句

在词级水平处理文本通常假定能够将文本划分成单个句子。正如我们已经看到，一些语料库已经提供在句子级别的访问。在下面的例子中，我们计算布朗语料库中每个句子的平均词数：

In [1]:
import nltk
len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())

20.250994070456922

在其他情况下，文本可能只是作为一个字符流。在将文本分词之前，我们需要将它分割
成句子。NLTK 通过包含`Punkt` 句子分割器(Kiss & Strunk, 2006)简化了这些。这里是使用它为一篇小说文本断句的例子。（请注意，如果在你读到这篇文章时分割器内部数据已经更新过，你会看到不同的输出。)

In [6]:
from pprint import pprint


sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt')
sents = sent_tokenizer.tokenize(text)
pprint(sents[171:181])

['In the wild events which were to follow this girl had no\n'
 'part at all; he never saw her again until all his tale was over.',
 'And yet, in some indescribable way, she kept recurring like a\n'
 'motive in music through all his mad adventures afterwards, and the\n'
 'glory of her strange hair ran like a red thread through those dark\n'
 'and ill-drawn tapestries of the night.',
 'For what followed was so\nimprobable, that it might well have been a dream.',
 'When Syme went out into the starlit street, he found it for the\n'
 'moment empty.',
 'Then he realised (in some odd way) that the silence\n'
 'was rather a living silence than a dead one.',
 'Directly outside the\n'
 'door stood a street lamp, whose gleam gilded the leaves of the tree\n'
 'that bent out over the fence behind him.',
 'About a foot from the\n'
 'lamp-post stood a figure almost as rigid and motionless as the\n'
 'lamp-post itself.',
 'The tall hat and long frock coat were black; the\n'
 'face, in an abrupt shadow

请注意，这个例子其实是一个单独的句子，报道Lucian Gregory先生的演讲。然而， 引用的演讲包含几个句子，这些已经被分割成几个单独的字符串。这对于大多数应用程序都是合理的行为。

断句是困难的，因为句号会被用来标记缩写而另一些句号同时标记缩写和句子结束，就像发生在缩写如“U.S.A.”上的那样。断句的另一种方法见 6.2节。

#### 分词

In [10]:
# 例3-2. 从分词表示字符串seg1和seg2中重建文本分词。
# seg1和seg2表示假设的一些儿童讲话的初始和最终分词。
# 函数segment()可以使用它们重现分词的文本。
def segment(text, segs):
    words = []
    last = 0
    for i in range(len(segs)):
        if segs[i] == '1':
            words.append(text[last:i + 1])
            last = i + 1
    words.append(text[last:])
    return words

In [11]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
segment(text, seg1)

['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']

In [12]:
segment(text, seg2)

['do',
 'you',
 'see',
 'the',
 'kitty',
 'see',
 'the',
 'doggy',
 'do',
 'you',
 'like',
 'the',
 'kitty',
 'like',
 'the',
 'doggy']

现在分词的任务变成了一个搜索问题：找到将文本字符串正确分割成词汇的字位串。我们假定学习者接收词，并将它们存储在一个内部词典中。给定一个合适的词典，是能够由词典中的词的序列来重构源文本的。读过(Brent & Cart-wright, 1995)之后，我们可以定义一 个目标函数，一个打分函数，我们将基于词典的大小和从词典中重构源文本所需的信息量尽 力优化它的值。我们在图 3-6 中说明了这些。
![3-6](./imgs/3-6.jpg)
实现这个目标函数是很简单的，如例子 3-3 所示。

In [13]:
def evaluate(text, segs):
    words = segment(text, segs)
    text_size = len(words)
    lexicon_size = len(' '.join(list(set(words))))
    return text_size + lexicon_size

In [14]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
seg3 = "0000100100000011001000000110000100010000001100010000001"
segment(text, seg3)

['doyou',
 'see',
 'thekitt',
 'y',
 'see',
 'thedogg',
 'y',
 'doyou',
 'like',
 'thekitt',
 'y',
 'like',
 'thedogg',
 'y']

In [15]:
evaluate(text, seg3)

46

In [16]:
evaluate(text, seg2)

47

In [17]:
evaluate(text, seg1)

63

最后一步是寻找最大化目标函数值的0和1的模式，例3-4中所示。请注意，最好的分
词包括像“thekitty”这样的“词”，因为数据中没有足够的证据进一步分割这个词。

In [20]:
# 例3-4. 使用模拟退火算法的非确定性搜索：一开始仅搜索短语分词；随机扰动0和1，
# 它们与“温度”成比例；每次迭代温度都会降低，扰动边界会减少。
from random import randint
def flip(segs, pos):
    return segs[:pos] + str(1 - int(segs[pos])) + segs[pos+1:]

def flip_n(segs, n):
    for i in range(n):
        segs = flip(segs, randint(0, len(segs)-1))
    return segs

def anneal(text, segs, iterations, cooling_rate):
    temperature = float(len(segs))
    while temperature > 0.5:
        best_segs, best = segs, evaluate(text, segs)
        for i in range(iterations):
            guess = flip_n(segs, int(round(temperature)))
            score = evaluate(text, guess)
            if score < best:
                best, best_segs = score, guess
        score, segs = best, best_segs
        temperature = temperature / cooling_rate
        print(evaluate(text, segs), segment(text, segs))
    print()
    return segs


In [21]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
anneal(text, seg1, 5000, 1.2)

63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
63 ['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
62 ['doy', 'ousee', 'th', 'ekitty', 'se', 'e', 'thedoggy', 'd', 'oyoulike', 'th', 'ekitty', 'like', 'thedoggy']
62 ['doy', 'ousee', 'th', 'ekitty', 'se', 'e', 'thedoggy', 'd', 'oyoulike', 'th', 'ekitty', 'like', 'thedoggy']
62 ['doy', 'ousee', 'th', 'ekitty', 'se', 'e', 'thedoggy', 'd', 'oyoulike', 'th', 'ekitty', 'like', 'thedoggy']
60 ['doyousee', 'thekitty', 'see', 'th', 'edoggy', 'd', 'o', 'youlike', 'thekitty', 'like', 'th', 'edoggy']
58 ['doyousee', 'thekitt

'0000100100000001001000000010000100010000000100010000000'

有了足够的数据，就可能以一个合理的准确度自动将文本分割成词汇。**这种方法可用于为那些词的边界没有任何视觉表示的书写系统分词**。


### 3.9 格式化：从链表到字符串

我们经常会写程序输出一个单独的数据项，例如一个语料库中满足一些复杂的标准的特定的元素，或者一个单独的总数统计，例如一个词计数器或一个标注器的性能。更多的时候， 我们写程序来产生一个结构化的结果；例如：一个数字或语言形式的表格，或原始数据的格式变换。当要表示的结果是语言时，文字输出通常是最自然的选择。然而当结果是数值时，可能最好是图形输出。在本节中，你将会学到呈现程序输出的各种方式。

#### 从链表到字符串

我们用于文本处理的最简单的一种结构化对象是词链表。当我们希望把这些输出到显器或文件时，必须把这些词的链表转换成字符串。在Python 做这些，我们使用的 `join()`方法，并指定作为“胶水”使用的字符串：

In [1]:
silly = ['We', 'called', 'him', 'Tortoise', 'because', 'he', 'taught', 'us', '.']

In [2]:
' '.join(silly)

'We called him Tortoise because he taught us .'

In [3]:
';'.join(silly)

'We;called;him;Tortoise;because;he;taught;us;.'

In [4]:
''.join(silly)

'WecalledhimTortoisebecausehetaughtus.'

#### 字符串与格式

In [6]:
word = 'cat'
sentence = """hello
world
"""
print(sentence)

hello
world



In [7]:
sentence

'hello\nworld\n'

还有许多其他有用的方法来将一个对象作为字符串显示。这可能是为了人阅读的方便， 或是因为我们希望导出我们的数据到一个特定的能被外部程序使用的文件格式。 格式化输出通常包含变量和预先指定的字符串的一个组合。例如：给定一个频率分布 `fdist`，我们可以这样做：

In [9]:
import nltk
fdist = nltk.FreqDist(['dog', 'cat', 'dog', 'cat', 'dog', 'snake', 'dog', 'cat'])
for word in fdist:
    print(word, '->', fdist[word], ';')

dog -> 4 ;
cat -> 3 ;
snake -> 1 ;


In [10]:
# 字符串格式化表达式
for word in fdist:
    print('%s->%d;' % (word, fdist[word]))

dog->4;
cat->3;
snake->1;


特殊符号`%s` 和`%d`是字符串和整数（十进制数）的占位符。我们可以将这些嵌入在一个字符串中，然后使用`%`操作符把它们组合起来。让我们更深入的解开这段代码，以便更仔细的观察它的行为：

In [11]:
'%s->' % 'cat'

'cat->'

In [12]:
'%d' % 3

'3'

In [13]:
'I want a %s right now' % 'coffee'

'I want a coffee right now'

In [14]:
"%s wants a %s %s" % ("Lee", "sandwich", "for lunch")

'Lee wants a sandwich for lunch'

In [15]:
template = 'Lee wants %s right now'
menu = ['sandwich', 'spam fritter', 'pancake']
for snack in menu:
    print(template % snack)

Lee wants sandwich right now
Lee wants spam fritter right now
Lee wants pancake right now


#### 排列

到目前为止，我们的格式化字符串可以在页面（或屏幕）上输出任意的宽度，如`%s`和
%d。我们也可以指定宽度，如`%6s`，产生一个宽度为`6`的字符串。缺省情况它是右对齐的，但我们可以包括一个减号使它左对齐。我们事先不知道要显示的值应该有多宽时， 可以在格式化字符串中用`*`替换宽度值，然后指定一个变量。

In [16]:
'%6s' % 'dog'

'   dog'

In [17]:
'%-6s' % 'dog'

'dog   '

In [18]:
width = 6
'%-*s' % (width, 'dog')

'dog   '

其他控制字符用于十进制整数和浮点数。因为百分号`%`在格式化字符串中有特殊解释， 我们要在它前面加另一个`%`才能输出它。

In [19]:
count, total = 3205, 9375
"accuracy for %d words: %2.4f%%" % (total, 100 * count / total)

'accuracy for 9375 words: 34.1867%'

格式化字符串的一个重要用途是**用于数据制表**。回想一下，在 2.1节中，我们看到从条件频率分布中制表的数据。让我们自己来制表，行使对标题和列宽的完全控制，如例 3-5所示。注意语言处理工作与结果制表之间是明确分离的。

In [28]:
# 例3-5. 布朗语料库的不同部分的频率模型
def tabulate(cfdist, words, categories):
    print('%-16s' % 'Category', end=' ')
    for word in words:
        print('%6s' % word, end=' ')
    print()
    for category in categories:
        print('%-16s' % category, end=' ')
        for word in words:
            print('%6d' % cfdist[category][word], end=' ')
        print()

In [29]:
from nltk.corpus import brown
cfd = nltk.ConditionalFreqDist(
    (genre, word)
    for genre in brown.categories()
    for word in brown.words(categories=genre))
genres = ['news', 'religion', 'hobbies', 'science_fiction', 'romance', 'humor']
modals = ['can', 'could', 'may', 'might', 'must', 'will']
tabulate(cfd, modals, genres)

Category            can  could    may  might   must   will 
news                 93     86     66     38     50    389 
religion             82     59     78     12     54     71 
hobbies             268     58    131     22     83    264 
science_fiction      16     49      4     12      8     16 
romance              74    193     11     51     45     43 
humor                16     30      8      8      9     13 


In [30]:
# 回顾在例 3-1 列出的，我们使用了格式化字符串“%*s”。
# 这使我们可以使用变量指定一个字段的宽度。
# 我们可以使用width = max(len(w) for w in words)自动定制列的宽度，
# 使其足够容纳所有的词
# 要记住print 语句结尾处的逗号增加了一个额外的空格，
# 这样能够防止列标题相互重叠。
'%*s' % (15, "Monty Python")

'   Monty Python'

#### 将结果写入文件

我们已经看到了如何读取文本文件（3.1节）。将输出写入文件往往也很有用。下面的
代码打开可写文件 `output.txt`，将程序的输出保存到文件。

In [32]:
output_file = open("output.txt", 'w')
words = set(nltk.corpus.genesis.words('english-kjv.txt'))
for word in sorted(words):
    output_file.write(word + "\r\n")

当我们 **将非文本数据写入文件时，我们必须先将它转换为字符串。正如我们前面所看到
的，可以使用格式化字符串来做这一转换** 。关闭文件之前，让我们把总词数写入我们的文件。

In [33]:
len(words)

2789

In [34]:
str(len(words))

'2789'

In [35]:
output_file.write(str(len(words)) + "\n")
output_file.close()

#### 文本换行
我们可以在Python 的 `textwrap` 模块的帮助下采取换行。为了最大程度的清晰，我们 将每一个步骤分在一行

In [37]:
saying = ['After', 'all', 'is', 'said', 'and', 'done', ',', 
          'more', 'is', 'said', 'than', 'done', '.']
from textwrap import fill
format = '%s_(%d),'
pieces = [format % (word, len(word)) for word in saying]
output = ' '.join(pieces)
wrapped = fill(output)
print(wrapped)

After_(5), all_(3), is_(2), said_(4), and_(3), done_(4), ,_(1),
more_(4), is_(2), said_(4), than_(4), done_(4), ._(1),


### 3.10 小结

- 我们可以使用`text = open(f).read()`从一个文件`f`读取文本。
- 词形归并是一个过程，将一个词的各种形式（如：appeared，appears）映射到这个词标准的或引用的形式，也称为词位或词元（如：appear）。
- 如果一个正则表达式字符串包含一个反斜杠，你应该使用原始字符串与一个`r`前缀：`r'regexp'`，告诉 Python不要预处理这个字符串。